In [2]:
import pandas as pd

from src.algorithms.alternating_least_squares import AlternatingLeastSquares
from src.helpers.dataset_indexer import DatasetIndexer
from src.helpers.checkpoint_manager import CheckpointManager
from src.recommenders import CollaborativeFilteringRecommenderBuilder
from src.backends import Backend
from src.helpers._logging import logger  # noqa
from src.settings import settings
from src.utils import vocabulary_based_one_hot_encode

from src.helpers.graphing import (
    plot_als_train_test_loss_evolution,
    plot_als_train_test_rmse_evolution,
    # plot_error_evolution,
    plot_power_low_distribution,
    plot_data_item_distribution_as_hist,
)

In [3]:
USER_HEADER = "userId"
ITEM_HEADER = "movieId"
RATING_HEADER = "rating"
FEATURE_TO_ENCODE = "genres"
ITEM_FEATURE_LIST = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "IMAX",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

In [4]:
dataset_indexer = DatasetIndexer(
    file_path="./ml-32m/ratings.csv",
    user_header=USER_HEADER,
    item_header=ITEM_HEADER,
    rating_header=RATING_HEADER,
    limit=settings.general.LINES_COUNT_TO_READ,
)

indexed_data = dataset_indexer.index_simple(
    approximate_train_ratio=settings.general.APPROXIMATE_TRAIN_RATIO
)

2025-01-09 02:52:37,333 [WARNING] The current implementation does not split the data into train and test sets exactly with the provided ratio. We use the provided ratio as a probability for a Bernoulli distribution to know whether a given data point should be used as a training data or a test data.
2025-01-09 02:52:44,887 [WARNING] The limit of lines (.i.e 1000000) to index has been reached. Exiting without loading the rest... 
2025-01-09 02:52:44,888 [INFO] Successfully indexed 1000000 lines from ./ml-32m/ratings.csv


In [5]:
# Import the movie csv file that will act as our movie database
# And that database is needed by the backend to query the movies
item_database = (
    pd.read_csv("./ml-32m/movies.csv", dtype={ITEM_HEADER: str})
    .assign(
        genres=lambda df: df[FEATURE_TO_ENCODE].apply(lambda genres: genres.split("|")),
        feature_vector=lambda df: df[FEATURE_TO_ENCODE].apply(
            lambda g: vocabulary_based_one_hot_encode(
                words=g, vocabulary=ITEM_FEATURE_LIST
            )
        ),
    )
    .set_index(ITEM_HEADER)  # Set the movieId as the index
    .to_dict(orient="index")  # Convert the DataFrame to a dictionary
)

In [6]:
# plot_data_item_distribution_as_hist(indexed_data)

In [7]:
# plot_power_low_distribution(indexed_data,)

In [8]:
als_instance = AlternatingLeastSquares(
    hyper_lambda=settings.als.HYPER_LAMBDA,
    hyper_gamma=settings.als.HYPER_GAMMA,
    hyper_tau=settings.als.HYPER_TAU,
    hyper_n_epochs=settings.als.HYPER_N_EPOCH,
    hyper_n_factors=settings.als.HYPER_N_FACTOR,
)

als_backend = Backend(
    # Define the algorithm
    algorithm=als_instance,
    checkpoint_manager=CheckpointManager(
        checkpoint_folder=settings.als.CHECKPOINT_FOLDER,
        sub_folder=str(settings.general.LINES_COUNT_TO_READ),
    ),
    # The predictor needs this to render the name of the items
    item_database=item_database,
    # Whether we should resume by using the last state of
    # the algorithm the checkpoint manager folder or not.
    resume=True,
    save_checkpoint=False,
)

In [9]:
recommender_builder = CollaborativeFilteringRecommenderBuilder(
    backend=als_backend,
)

# This might take some moment before finishing
recommender = recommender_builder.build(data=indexed_data)

2025-01-09 02:52:46,161 [INFO] Starting the build of the recommender using AlternatingLeastSquares with the state {'hyper_lambda': 0.1, 'hyper_tau': 0.1, 'hyper_gamma': 0.01, 'hyper_n_epochs': 20, 'hyper_n_factors': 10, 'user_factors': None, 'item_factors': None, 'user_biases': None, 'item_biases': None, 'loss_train': [], 'loss_test': [], 'rmse_train': [], 'rmse_test': []}
2025-01-09 02:52:46,163 [INFO] Starting a model fitting using the backend AlternatingLeastSquares...
2025-01-09 02:52:46,167 [INFO] Checkpoint ./artifacts/checkpoints/als/1000000/20250107-223930_lambda5_gamma0.2_tau0.5_n_epochs10_n_factors10.pkl loaded with success
2025-01-09 02:52:46,168 [INFO] All factors and biases are already provided, so no initialization is needed.
2025-01-09 02:52:46,170 [ERROR] Cannot train the model more because hyperparameter 'hyper_n_epochs' (10) is already greater or equal to the final number of epochs wanted which is 10. Please check the value of 'hyper_n_epochs' and adjust accordingly. 

In [10]:
# plot_als_train_test_rmse_evolution(als_backend.algorithm)

In [11]:
# plot_als_train_test_loss_evolution(als_backend.algorithm)

In [12]:
#
prediction_input = [("17", 4)]
recommender.recommend(prediction_input)

Hé predictio iter
Hé predictio iter
Hé predictio iter
Hé predictio iter


[{'title': 'Aziz Ansari: Live at Madison Square Garden (2015)',
  'genres': ['Comedy'],
  'feature_vector': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'title': 'Zeitgeist: Moving Forward (2011)',
  'genres': ['Documentary'],
  'feature_vector': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'title': 'Legend, The (Legend of Fong Sai-Yuk, The) (Fong Sai Yuk) (1993)',
  'genres': ['Action', 'Comedy'],
  'feature_vector': array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'title': 'Woman of the Year (1942)',
  'genres': ['Comedy', 'Romance'],
  'feature_vector': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])},
 {'title': 'Zeitgeist: The Movie (2007)',
  'genres': ['Documentary', 'War'],
  'feature_vector': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])},
 {'title': 'Limbo (1999)',
  'genres': ['Drama'],
  'feature_vector': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},


In [13]:
prediction_input = [("267654", 4)]  # Harry Poter
recommender.recommend(prediction_input)

Hé predictio iter
Hé predictio iter
Hé predictio iter
Hé predictio iter


[{'title': 'Aziz Ansari: Live at Madison Square Garden (2015)',
  'genres': ['Comedy'],
  'feature_vector': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'title': 'The Yakuza Papers, Vol. 4: Police Tactics (1974)',
  'genres': ['Crime', 'Drama'],
  'feature_vector': array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'title': 'The Last House on the Beach (1978)',
  'genres': ['Crime', 'Drama', 'Horror', 'Thriller'],
  'feature_vector': array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'title': 'Evilenko (2004)',
  'genres': ['Crime', 'Horror', 'Thriller'],
  'feature_vector': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'title': 'Afflicted, The (2010)',
  'genres': ['Horror', 'Thriller'],
  'feature_vector': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'title': 'Paradise (1982)',
  'genres': ['Adventure', 'Romance'],
  'feature_vector': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [14]:
#
recommender.recommend()

[{'title': 'Aziz Ansari: Live at Madison Square Garden (2015)',
  'genres': ['Comedy'],
  'feature_vector': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'title': 'Facing the Giants (2006)',
  'genres': ['Action', 'Drama'],
  'feature_vector': array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'title': 'The Last House on the Beach (1978)',
  'genres': ['Crime', 'Drama', 'Horror', 'Thriller'],
  'feature_vector': array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'title': 'The Yakuza Papers, Vol. 4: Police Tactics (1974)',
  'genres': ['Crime', 'Drama'],
  'feature_vector': array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'title': 'Evilenko (2004)',
  'genres': ['Crime', 'Horror', 'Thriller'],
  'feature_vector': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'title': 'Afflicted, The (2010)',
  'genres': ['Horror', 'Thriller'],
  'feature_vector': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 